In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install multi_key_dict
!pip install word2number

  Created wheel for multi-key-dict: filename=multi_key_dict-2.0.3-cp36-none-any.whl size=9300 sha256=45e83379a6715539349fdf86cf0c0319a09e1d1ef69b73fc2889df30669d6458
  Stored in directory: /root/.cache/pip/wheels/1d/26/5b/ac32658fddc88f045a516406ff69cafabaab2602eb96bbe2f0
Successfully built multi-key-dict
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=1a22b0fea34b8137735bc9d794260f72ef696c90d7be5c1cd43db1bd5e8d329a
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number


In [0]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import argparse
from collections import Counter
import json
import os
from tqdm import *
import numpy as np
import re
import random
# allows multiple keys for one key-value pair
from multi_key_dict import multi_key_dict
from word2number import w2n
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
DATA_PATH = "/content/drive/My Drive/Intro to AI Project/data/"

##Vocabulary Wrapper

In [0]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word, idx=None):
        if idx is None:
            if not word in self.word2idx:
                self.word2idx[word] = self.idx
                self.idx2word[self.idx] = word
                self.idx += 1
            return self.idx
        else:
            if not word in self.word2idx:
                self.word2idx[word] = idx
                if idx in self.idx2word.keys():
                    self.idx2word[idx].append(word)
                else:
                    self.idx2word[idx] = [word]

                return idx

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<pad>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)

##Rules for ingredients cleaning

In [0]:

lemmatizer = WordNetLemmatizer()
#\d: decimal; \s: whitespace; 
# pattern of units
num_units = re.compile(r'(\d)+(\s)?(cubes?|cups?|c(a?ns?)?|[tT]|bulbs?|sprigs?|glass(es)?|dice|blocks?|an?|l|fl(uid)?\.?|ears?|lea(f|ves)|jars?|cartons?|strips?|heads?|wedges?|envelopes?|pints?|stalks?|sticks?|pinch(es)?|qts?|quarts?|handful|weight|bottles?|grinds?|tb\.?|lbs?\.?|oz\.?|mls?|g|cloves?|containers?|tablespoons?|teaspoons?|dash(es)?|pounds?|pinch|box(es)?|cans?|(milli)?lit[er]{2}s?|pkg\.?|pack(et)s?|packages?|whole|bars?|bags?|tbsps?\.?|tbs\.?|ts|tsps?\.?|ounces?|dash|pieces?|slices?|bunch(es)?|sticks?|fl\.?|gallons?|squares?|knobs?|grams?|kgs?|tub(es)?|kilograms?|tins?|%|drizzles?|splash(es)?|chunks?|inch(es)?)(\s|$)')
   
replace_dict_ingrs = multi_key_dict()
replace_dict_ingrs['&',"'n"] = ' and '
replace_dict_ingrs['-'] = ' '

# preprocess text for ingredients 
def get_ingredient(det_ingr):
    det_ingr = det_ingr['text'].lower()
    re.sub("[\(\[\{].*?[\)\]\}]", "", det_ingr) #remove text in parentheses
    re.sub(num_units,"",det_ingr) #remove units
    for k_list in replace_dict_ingrs.keys():
      for k in k_list:
        if k in det_ingr:
          det_ingr = det_ingr.replace(k,replace_dict_ingrs[k]) #replace characters like (& n -)        
    cleaned_list = []
    for word in det_ingr.split():
      if word.isalpha():  # word only, no numbers or other symbols
        cleaned_list.append(lemmatizer.lemmatize(word)) 
    det_ingr_cleaned = " ".join(cleaned_list)
    det_ingr_cleaned = det_ingr_cleaned.replace(' ', '_')
    return det_ingr_cleaned

get_ingredient({"text":"1g salt salmon 100g tomato"})


'salt_salmon_tomato'

##Rules for instructions cleaning

In [0]:
# Interesting, using multi_key_dict to reduce dimensionality/standardize

replace_dict_instrs = multi_key_dict()
replace_dict_instrs[' nonfat ', ' non-fat '] = ' fat-free '
replace_dict_instrs['low-fat', 'reduced-fat'] = ' fat-reduced '
replace_dict_instrs[' flatleaf '] = ' flat-leaf '
replace_dict_instrs['&',"'n"] = ' and '
replace_dict_instrs['!','?'] = '.'
replace_dict_instrs[';',':'] = '.'
replace_dict_instrs['-','(',')',"[",']',"#","$"] = ' '
replace_dict_instrs[' a '," an "] = ' 1 '

# unit conversions: standardize units
unit_conversions = multi_key_dict()
unit_conversions['pounds', 'pound', 'lbs', 'lb'] = ('g', 453.6)
unit_conversions['ounces', 'ounce', 'ozs', 'oz', 'weight'] = ('g', 28.35)
unit_conversions['can', 'cans', 'cn'] = ('can', 1)
unit_conversions['pints', 'pint', 'pts', 'pt'] = ('l', 0.4732)
unit_conversions['quarts', 'quart', 'qts', 'qt'] = ('l', 1.946352946)
unit_conversions['cups', 'cup', 'c'] = ('l', 0.2366)
unit_conversions['cubes', 'cube'] = ('cube', 1)
unit_conversions['fluid', 'fl'] = ('l', 0.02957)
unit_conversions['tablespoons', 'tablespoon', 'tbsps', 'tbsp', 'tb', 'tbs', 'T'] = ('l', 0.01479)
unit_conversions['teaspoons', 'teaspoon', 'tsps', 'tsp', 't', 'ts'] = ('l', 0.004929)
unit_conversions['milliliters', 'millilitres', 'ml'] = ('l', 0.001)
unit_conversions['gram', 'gs', 'grams'] = ('g', 1)
unit_conversions['kilogram', 'kgs', 'kg', 'kilograms'] = ('g', 0.001)

def get_instruction(instruction, instruction_mode=True):
    instruction = instruction.lower()
    re.sub("[\(\[\{].*?[\)\]\}]", "", instruction) #remove text in parentheses
    for k_list in replace_dict_instrs.keys():
      for k in k_list:
        if k in instruction:
          instruction = instruction.replace(k,replace_dict_instrs[k]) # use rules to standardize labels
    instruction = instruction.replace('half','1/2') # should it be inluded above?
    cleaned_list = []
    # unit conversion
    words = instruction.split()
    for i in range (len(words)):
      if words[i] in unit_conversions:
        try:
          # the word before unit is most likely a number
          num = w2n.word_to_num(words[i-1])
          # unit conversion
          num = int(num/unit_conversions[words[i]][1])
          cleaned_list[-1] = str(num)
          cleaned_list.append(unit_conversions[words[i]][0])
          continue
        except:
          pass
        try:
          num = words[i-1]
          if '/' in num:
            num = float(num.split('/')[0])/float(num.split('/')[1])
          else:
            num = float(num)
          num = num/unit_conversions[words[i]][1]
          cleaned_list[-1] = str(num)
          cleaned_list.append(unit_conversions[words[i]][0])
          continue
        except:
          pass
      else:
        cleaned_list.append(words[i])
                                      
    instruction = " ".join(cleaned_list)
    # instruction too short to be valid
    if len(instruction) < 3:  
      return ""
    # Remove "1." or "2." for steps -> for model  
    if len(instruction) > 0 and instruction[0].isdigit() and instruction[1] == '.' and instruction_mode:
        instruction = instruction[2:].strip()
    return instruction
 
print (get_instruction("1.Arrange 3/5 kg veggies on top of two kg salmon 'n seal foil tightly to create a tightly sealed pocket."))
print (get_instruction("1."))

arrange 600.0 g veggies on top of 2000 g salmon and seal foil tightly to create 1 tightly sealed pocket.



# Cluster similar ingredients

In [0]:
def cluster_ingredients(counter_ingrs):
    mydict = dict()
    mydict_ingrs = dict()

    for k, v in counter_ingrs.items():  #counter_ingrs is a dict

        w1 = k.split('_')[-1] #last word
        w2 = k.split('_')[0]  #first word
        lw = [w1, w2]
        if len(k.split('_')) > 1:
            w3 = k.split('_')[0] + '_' + k.split('_')[1] #first two words
            w4 = k.split('_')[-2] + '_' + k.split('_')[-1] #last two words

            lw = [w1, w2, w4, w3]

        gotit = 0
        for w in lw:
            if w in counter_ingrs.keys():
                # check if its parts are
                parts = w.split('_')
                if len(parts) == 2:
                    if parts[0] in counter_ingrs.keys():
                        w = parts[0]
                    elif parts[1] in counter_ingrs.keys():
                        w = parts[1]
                if w in mydict.keys():
                    mydict[w] += v
                    mydict_ingrs[w].append(k)
                else:
                    mydict[w] = v
                    mydict_ingrs[w] = [k]
                gotit = 1
                break
        if gotit == 0:
            mydict[k] = v
            mydict_ingrs[k] = [k]

    return mydict, mydict_ingrs


def update_counter(list_, counter_toks, istrain=False):
    for sentence in list_:
        tokens = nltk.tokenize.word_tokenize(sentence)
        if istrain:
            counter_toks.update(tokens)

In [0]:
def build_vocab_recipe1m():
    print ("Loading data...")
    dets = json.load(open(os.path.join(DATA_PATH, 'det_ingrs.json'), 'r'))
    layer1 = json.load(open(os.path.join(DATA_PATH, 'layer1.json'), 'r'))

    print("Loaded data.")
    print("Found %d recipes in the dataset." % (len(layer1)))
    
    # dict with id as key and number as value
    idx2ind = {}
    for i, entry in enumerate(dets):
        idx2ind[entry['id']] = i

    # creating new file paths
    ingrs_file = DATA_PATH + 'allingrs_count.pkl'
    instrs_file = DATA_PATH + 'allwords_count.pkl'
    

    #####
    # 1. Count words in dataset and clean
    #####
    
    # if file exist then load files
    if os.path.exists(ingrs_file) and os.path.exists(instrs_file):
        print ("loading pre-extracted word counters")
        counter_ingrs = pickle.load(open(DATA_PATH + 'allingrs_count.pkl', 'rb'))
        counter_toks = pickle.load(open(DATA_PATH + 'allwords_count.pkl', 'rb'))
    else:
        counter_toks = Counter()
        counter_ingrs = Counter()

        for i, entry in tqdm(enumerate(layer1)):

            # get all instructions for this recipe
            instrs = entry['instructions']

            instrs_list = []
            ingrs_list = []

            # retrieve pre-detected ingredients for this entry
            det_ingrs = dets[idx2ind[entry['id']]]['ingredients']
            #print (det_ingrs)
            valid = dets[idx2ind[entry['id']]]['valid']

            for j, det_ingr in enumerate(det_ingrs):
                if len(det_ingr) > 0 and valid[j]:
                    #print (det_ingr)
                    det_ingr_undrs = get_ingredient(det_ingr)
                    #print (det_ingr_undrs)
                    if len(det_ingr_undrs):
                      ingrs_list.append(det_ingr_undrs)

            # get raw text for instructions of this entry
            acc_len = 0
            for instr in instrs:
                instr = instr['text']
                instr = get_instruction(instr)
                if len(instr) > 0:
                    instrs_list.append(instr)
                    acc_len += len(instr)

            # discard recipes with too few or too many ingredients or instruction words
            if len(ingrs_list) < 2 or len(instrs_list) < 2 \
                    or len(instrs_list) >= 20 or len(ingrs_list) >= 20 \
                    or acc_len < 20:
                continue

            # tokenize sentences and update counter
            update_counter(instrs_list, counter_toks, istrain=entry['partition'] == 'train')
            title = nltk.tokenize.word_tokenize(entry['title'].lower())
            if entry['partition'] == 'train':
                counter_toks.update(title)
            if entry['partition'] == 'train':
                counter_ingrs.update(ingrs_list)
        pickle.dump(counter_ingrs, open(ingrs_file, 'wb'))
        pickle.dump(counter_toks, open(instrs_file, 'wb'))


    counter_ingrs, cluster_ingrs = cluster_ingredients(counter_ingrs)

    # If the word frequency is less than 'threshold', then the word is discarded.
    words = [word for word, cnt in counter_toks.items() if cnt >= 10]
    ingrs = {word: cnt for word, cnt in counter_ingrs.items() if cnt >= 3}

    # Recipe vocab
    # Create a vocab wrapper and add some special tokens.
    vocab_toks = Vocabulary()
    vocab_toks.add_word('<start>')
    vocab_toks.add_word('<end>')
    vocab_toks.add_word('<eoi>')

    # Add the words to the vocabulary.
    for i, word in enumerate(words):
        vocab_toks.add_word(word)
    vocab_toks.add_word('<pad>')

    # Ingredient vocab
    # Create a vocab wrapper for ingredients
    vocab_ingrs = Vocabulary()
    idx = vocab_ingrs.add_word('<end>')
    # this returns the next idx to add words to
    # Add the ingredients to the vocabulary.
    for k, _ in ingrs.items():
        for ingr in cluster_ingrs[k]:
            idx = vocab_ingrs.add_word(ingr, idx)
        idx += 1
    _ = vocab_ingrs.add_word('<pad>', idx)
    for k,v in vocab_ingrs.idx2word.items():
        if type(v) == list:
            v.sort(key = lambda x: len(x))

    print("Total ingr vocabulary size: {}".format(len(vocab_ingrs)))
    print("Total token vocabulary size: {}".format(len(vocab_toks)))
    dataset = {'train': [], 'val': [], 'test': []}

    ######
    # 2. Tokenize and build dataset based on vocabularies.
    ######
    for i, entry in tqdm(enumerate(layer1)):

        # get all instructions for this recipe
        instrs = entry['instructions']

        instrs_list = []
        ingrs_list = []

        # retrieve pre-detected ingredients for this entry
        det_ingrs = dets[idx2ind[entry['id']]]['ingredients']
        valid = dets[idx2ind[entry['id']]]['valid']
        labels = []

        for j, det_ingr in enumerate(det_ingrs):
            if len(det_ingr) > 0 and valid[j]:
                det_ingr_undrs = get_ingredient(det_ingr)
                if len(det_ingr_undrs)>0:
                    if det_ingr_undrs in vocab_ingrs.word2idx:
                        ingrs_list.append(vocab_ingrs.idx2word[vocab_ingrs.word2idx[det_ingr_undrs]][0])
                    else:
                        ingrs_list.append(det_ingr_undrs)
                label_idx = vocab_ingrs(det_ingr_undrs)
                if label_idx is not vocab_ingrs('<pad>') and label_idx not in labels:
                    labels.append(label_idx)

        # get raw text for instructions of this entry
        acc_len = 0
        for instr in instrs:
            instr = instr['text']
            instr = get_instruction(instr, replace_dict_instrs)
            if len(instr) > 0:
                acc_len += len(instr)
                instrs_list.append(instr)

        # we discard recipes with too many or too few ingredients or instruction words
        if len(labels) < 2 or len(instrs_list) < 2 \
                or len(instrs_list) >= 20 or len(labels) >= 20 \
                or acc_len < 20:
            continue

        # tokenize sentences
        toks = []

        for instr in instrs_list:
            tokens = nltk.tokenize.word_tokenize(instr)
            toks.append(tokens)

        title = get_instruction(entry['title'])

        newentry = {'id': entry['id'], 'instructions': instrs_list, 'tokenized': toks,
                    'ingredients': ingrs_list, 'title': title}
        dataset[entry['partition']].append(newentry)

    print('Dataset size:')
    for split in dataset.keys():
        print(split, ':', len(dataset[split]))

    return vocab_ingrs, vocab_toks, dataset



In [0]:
vocab_ingrs, vocab_toks, dataset = build_vocab_recipe1m()
with open(os.path.join(DATA_PATH, 'recipe1m_vocab_ingrs.pkl'), 'wb') as f:
  pickle.dump(vocab_ingrs, f)
with open(os.path.join(DATA_PATH+'recipe1m_vocab_toks.pkl'), 'wb') as f:
  pickle.dump(vocab_toks, f)
for split in dataset.keys():
  with open(os.path.join(DATA_PATH+'recipe1m_' + split + '.pkl'), 'wb') as f:
    pickle.dump(dataset[split], f)


Loading data...
Loaded data.
Found 1029720 recipes in the dataset.
loading pre-extracted word counters


69it [00:00, 673.94it/s]

Total ingr vocabulary size: 1909
Total token vocabulary size: 21487


1029720it [26:06, 657.48it/s]


Dataset size:
train : 645108
val : 138744
test : 138099


In [0]:
print (vocab_ingrs.idx2word[vocab_ingrs.word2idx["mustard_powder"]])
print (vocab_ingrs.idx2word[vocab_ingrs.word2idx["cheese"]])
print (vocab_ingrs.idx2word[vocab_ingrs.word2idx["chicken"]])

['mustard', 'dry_mustard', 'hot_mustard', 'pub_mustard', 'dark_mustard', 'dijon_mustard', 'honey_mustard', 'dried_mustard', 'spicy_mustard', 'brown_mustard', 'sweet_mustard', 'cajun_mustard', 'meaux_mustard', 'maple_mustard', 'mustard_paste', 'salad_mustard', 'stout_mustard', 'vegan_mustard', 'mustard_powder', 'yellow_mustard', 'grainy_mustard', 'ground_mustard', 'creole_mustard', 'garlic_mustard', 'french_mustard', 'german_mustard', 'wasabi_mustard', 'english_mustard', 'dry_hot_mustard', 'chinese_mustard', 'pommery_mustard', 'hot_dry_mustard', 'swedish_mustard', 'prepared_mustard', 'honeycup_mustard', 'chipotle_mustard', 'sweet_hot_mustard', 'cranberry_mustard', 'dijonnaise_mustard', 'sweet_dark_mustard', 'head_honey_mustard', 'honey_dijon_mustard', 'dijon_style_mustard', 'spicy_brown_mustard', 'whole_grain_mustard', 'horseradish_mustard', 'hot_english_mustard', 'grey_poupon_mustard', 'chinese_hot_mustard', 'coarse_grain_mustard', 'prepared_hot_mustard', 'stone_ground_mustard', 'frenc

In [0]:
data = pickle.load(open(DATA_PATH + 'recipe1m_train.pkl', 'rb'))

In [0]:
i = 0
print (data[i]['id'])
print (data[i]['title'])
print (data[i]['ingredients'])
print (data[i]['instructions'])

000033e39b
dilly macaroni salad recipe
['macaroni', 'cheese', 'celery', 'pepper', 'pimento', 'mayonnaise', 'vinegar', 'salt', 'dill']
['cook macaroni according to package directions. drain well.', 'cold.', 'combine macaroni, cheese cubes, celery, green pepper and pimento.', 'blend together mayonnaise or possibly salad dressing, vinegar, salt and dill weed. add in to macaroni mix.', 'toss lightly.', 'cover and refrigeratewell.', 'serve salad in lettuce lined bowl if you like.', 'makes 6 servings.']


# Data Sampling

In [0]:
# 1/n of original data set (sample)
#!mkdir $DATA_PATH+'sample-100'
for split in ["train","val","test"]:
    with open(os.path.join(DATA_PATH+'sample-100/recipe1m_' + split + '.pkl'), 'wb') as f:
        data = pickle.load(open(DATA_PATH + 'recipe1m_%s.pkl'%split, 'rb'))
        data = random.sample(data,int(len(data)/100))
        print(split+':'+str(len(data)))
        pickle.dump(data, f)

train:6451
val:1387
test:1380


In [0]:
# 1/n of original data set (slicing)
!mkdir "$DATA_PATH""sample-20"
n = 20
for split in ["train","val","test"]:
    data = pickle.load(open(DATA_PATH + 'recipe1m_%s.pkl'%split, 'rb'))
    for i in range (n):
        with open(os.path.join(DATA_PATH+'sample-%d/recipe1m_'%n + split +'-'+ str(i)+'.pkl'), 'wb') as f:
            split_data = data[i*int(len(data)/n):(i+1)*int(len(data)/n)]
            print(split+'-'+str(i)+':'+str(len(split_data)))
            pickle.dump(split_data, f)

mkdir: cannot create directory ‘/content/drive/My Drive/Intro to AI Project/data/sample-20’: File exists
train-0:32255
train-1:32255
train-2:32255
train-3:32255
train-4:32255
train-5:32255
train-6:32255
train-7:32255
train-8:32255
train-9:32255
train-10:32255
train-11:32255
train-12:32255
train-13:32255
train-14:32255
train-15:32255
train-16:32255
train-17:32255
train-18:32255
train-19:32255
val-0:6937
val-1:6937
val-2:6937
val-3:6937
val-4:6937
val-5:6937
val-6:6937
val-7:6937
val-8:6937
val-9:6937
val-10:6937
val-11:6937
val-12:6937
val-13:6937
val-14:6937
val-15:6937
val-16:6937
val-17:6937
val-18:6937
val-19:6937
test-0:6904
test-1:6904
test-2:6904
test-3:6904
test-4:6904
test-5:6904
test-6:6904
test-7:6904
test-8:6904
test-9:6904
test-10:6904
test-11:6904
test-12:6904
test-13:6904
test-14:6904
test-15:6904
test-16:6904
test-17:6904
test-18:6904
test-19:6904
